# Conditional value at risk in robust portfolio

This tutorial shows how to construct and solve the Robust Portfolio management problem (Zhu and Fukushima 2009, https://pubsonline.informs.org/doi/abs/10.1287/opre.1080.0684). The optimal allocation is achieved by minimizing the Conditional Value at Risk (CVaR). The problem can be formulated as follows:
$$
\begin{align}
\min_{\mathbf{u},\mathbf{x}, \alpha}\; &\alpha + \frac{\mathbf{\pi}^T\mathbf{u}}{1-\beta} \nonumber \\

\text{s.t. } &u_k \geq -\mathbf{y_k^T}\mathbf{x} - \alpha \;\forall k\in \left\{ 1 \ldots s \right \} \nonumber \\

&\mathbf{u}\geq \mathbf{0} \nonumber \\

&\mathbf{\pi}\mathbf{Y}\mathbf{x} \geq \mu \nonumber \\

&\mathbf{x}_{lb} \leq \mathbf{x} \leq \mathbf{x}_{ub} \nonumber \\

&\mathbf{1}^T\mathbf{x} = w_0 \nonumber \\
\end{align}
$$

There are $s$ samples of $n$ stocks. The investment decision ia denoted by $\mathbf{x}\in\mathbb{R}^n$, and $\mathbf{u}\in\mathbb{R}^s$ is an auxilary variable. $\alpha\in\mathbb{R}$ is an optimization paramater. The worst case minimum overall return rate, the confidence level, and the budget investment are denoted by $\mu,\beta,w_0$, repsectively. The stock returns are denoted by $\mathbf{Y}\in\mathbb{R}^{s\times n}$, and $\mathbf{y}_k$ denotes the $k$-th row of $\mathbf{Y}$. The probabilities of samples are denoted by the vector $\mathbf{\pi}\in\mathbb{R}^s$, and belong to the following uncertainty set:
$$
\mathbf{\pi}\in \left\{ \mathbf{\pi}: \mathbf{\pi}=\mathbf{\pi}_0+\mathbf{\eta}, \mathbf{1}^T\mathbf{\eta}=1, \mathbf{\eta}_{lb}\leq \mathbf{\eta} \leq \mathbf{\eta}_{ub} \right\}
$$
where $\mathbf{\pi}_0\in\mathbb{R}^s$ is a nominal value.

In [1]:
import cvxpy as cp
import numpy as np
import pandas as pd
import yfinance as yf


from lropt import Box
from lropt.robust_problem import RobustProblem
from lropt.uncertain import UncertainParameter

np.random.seed(seed=1234)

We use the yfinance package to load sampled data of stock of 8 stocks, as shown in the code below:

In [2]:
stocks = ['JPM', 'AMZN', 'TSLA', 'AAPL', 'GOOG', 'ZM', 'META', 'MCD']   # Stock names
start = '2021-1-2'                                                      # Start date of historical data
end='2021-12-31'                                                        # End date of historical data

#Load the data
data = pd.DataFrame([])
for stock in stocks:
    each = yf.Ticker(stock).history(start=start, end=end)
    close = each['Close'].values
    returns = (close[1:] - close[:-1]) / close[:-1]
    data[stock] = returns

#Print the data
data

,JPM,AMZN,TSLA,AAPL,GOOG,ZM,META,MCD
0,0.005441,0.010004,0.007317,0.012364,0.007337,0.002361,0.007548,0.005994
1,0.046956,-0.024897,0.028390,-0.033661,-0.003234,-0.045506,-0.028269,-0.002270
2,0.032839,0.007577,0.079447,0.034123,0.029943,-0.005546,0.020622,0.004644
3,0.001104,0.006496,0.078403,0.008632,0.011168,0.020759,-0.004354,0.018351
4,0.014924,-0.021519,-0.078214,-0.023249,-0.022405,-0.034038,-0.040102,-0.007597
...,...,...,...,...,...,...,...,...
245,0.003574,0.000184,0.057619,0.003644,0.001317,-0.007663,0.014495,0.003812
246,0.005723,-0.008178,0.025248,0.022975,0.006263,-0.021967,0.032633,0.008610
247,0.003035,0.005844,-0.005000,-0.005767,-0.010914,-0.019580,0.000116,-0.001342
248,-0.000504,-0.008555,-0.002095,0.000502,0.000386,-0.010666,-0.009474,0.002277


We define the problem variables and constants:

In [3]:
y = data.values     # Stock data as an array
s, n = y.shape      # Sample size/Number of stocks

x_lb = np.zeros(n)  # Lower bounds of investment decisions
x_ub = np.ones(n)   # Upper bounds of investment decisions

beta = 0.95         # Confidence interval
w0 = 1              # Investment budget
mu = 0.001          # Target minimum expected return rate

eta_ub = 0.0001     # Upper bound of eta
eta_lb = -0.0001    # Lower bound of eta

The problem is formulated and solved in the block below, using the Box uncertainty set which is defined as follows:

Box uncertainty: $\{ u \mid\| Au + b\|_\infty \leq \rho \}$ 

- $\rho$ : float, optional  
  * Box scaling. Default 1.0.
- $A$ : np.array, optional
  * Scaling matrix for u. Default identity matrix.
- $b$ : np.array, optional
  * Relocation vector for u. Default None.

We directly add the inequality constraints on $\eta$ using the $\textit{lb},\textit{ub},\textit{sum\_eq}$ optional arguments, which read as follows:
$$
lb \leq \mathbf{u} \leq ub \nonumber \\

\mathbf{1}^T\mathbf{u} = sum\_eq \nonumber
$$

In [4]:
uncertainty_set = Box(rho=max(eta_ub, -eta_lb), lb=eta_lb, ub=eta_ub, sum_eq=0) # Box uncertainty set
eta = UncertainParameter(s, uncertainty_set=uncertainty_set)                    # Uncertain parameter, controls the probabilities of samples
pi = 1/s + eta                                                                  # Probabilities of samples
x = cp.Variable(n)                                                              # Optimization variables, investment decisions
u = cp.Variable(s)                                                              # Optimization auxiliary variables
alpha = cp.Variable()                                                           # Optimization auxiliary variable

objective = cp.Minimize(alpha + 1/(1-beta) * (pi@u))                            # Problem objective
constraints = [
                u >= -y@x - alpha,
                u >= 0,
                pi@(y@x) >= mu,
                x >= x_lb,
                x <= x_ub,
                cp.sum(x) == w0,
              ]                                                                 # Problem constraints
prob = RobustProblem(objective=objective, constraints=constraints)              # Define the problem using LROPT
prob.solve()                                                                    # Solve the problem using LROPT

0.01854118785617952